In [1]:
import matplotlib.pyplot as plt
import polars as pl

import numpy as np
import pandas as pd

import polars as pl
import seaborn as sns

%reload_ext autoreload
%autoreload 2

In [2]:
df_train = pl.read_parquet('../experiments/datasets/master_card_basic/train.parquet')
df_test = pl.read_parquet('../experiments/datasets/master_card_basic/test.parquet')

In [20]:
# df_train

In [37]:
from sklearn.linear_model import LogisticRegression


target = ['is_fraud']
X_feautures = ['amount', 'channel', 'currency', 'device']
categorical_columns = ['channel', 'currency', 'device']
X_feautures = ['amount', 'user_id', 'lat', 'long']
categorical_columns = ['user_id', ]

X_train, y_train = df_train[X_feautures], df_train[target]
X_train = X_train.with_columns(pl.col(c).to_physical() for c in categorical_columns)

X_test, y_test = df_test[X_feautures], df_test[target]
X_test = X_test.with_columns(pl.col(c).to_physical() for c in categorical_columns)

In [38]:
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.discriminant_analysis import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler


X_train

transformer = make_column_transformer(
    (StandardScaler(), ['amount']),
    (OneHotEncoder(sparse_output=True), categorical_columns)
)
pipe = make_pipeline(transformer, LogisticRegression(class_weight ='balanced'))
pipe.fit(X_train, y_train.to_numpy().flatten())

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['amount']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['user_id'])])),
                ('logisticregression',
                 LogisticRegression(class_weight='balanced'))])

In [ ]:
# dir(pipe[1].n_features_in_)
# pipe[1].n_features_in_

user_id
cat
"""U14804"""
"""U16634"""
"""U18005"""
"""U13690"""
"""U04642"""
…
"""U18350"""
"""U16994"""
"""U01637"""


In [39]:
y_hat = pipe.predict(X_test)
y_hat

array([1, 1, 1, ..., 1, 0, 0], shape=(83258,))

In [40]:
from sklearn.metrics import accuracy_score, average_precision_score, f1_score, precision_score, recall_score, roc_auc_score


{
    "accuracy": accuracy_score(y_test, y_hat),
    "auc": roc_auc_score(y_test, y_hat),
    "f1_score": f1_score(y_test, y_hat),
    'average_precision': average_precision_score(y_test, y_hat),
}


{'accuracy': 0.5569194551874895,
 'auc': np.float64(0.5027193825960562),
 'f1_score': 0.14582754468833936,
 'average_precision': np.float64(0.08694897252819744)}